In [1]:
from datasets import load_dataset,Audio
from transformers import WhisperForConditionalGeneration, WhisperProcessor
import torch
from evaluate import load
def evaluateModel(path,v=""):
    common_voice = load_dataset("mozilla-foundation/common_voice_11_0", "zh-HK", split="test")
    common_voice = common_voice.cast_column("audio", Audio(sampling_rate=16000))
    common_voice = common_voice.select(range(10))

    processor = WhisperProcessor.from_pretrained(path)
    model = WhisperForConditionalGeneration.from_pretrained("../fineTune/fineTue/"+path+"/checkpoint-4000"+v).to("cuda")



    def map_to_pred(batch):
        audio = batch["audio"]
        input_features = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt").input_features
        batch["reference"] = processor.tokenizer._normalize(batch['sentence'])

        with torch.no_grad():
            predicted_ids = model.generate(input_features.to("cuda"))[0]
        transcription = processor.decode(predicted_ids)
        batch["prediction"] = processor.tokenizer._normalize(transcription)
        return batch

    result = common_voice.map(map_to_pred)

    wer = load("wer")
    print(100 * wer.compute(references=result["reference"], predictions=result["prediction"]))
    print(result["reference"])
    print(result["prediction"])


In [2]:
whisper_path=["openai/whisper-tiny","openai/whisper-base","openai/whisper-small","openai/whisper-medium","openai/whisper-large","openai/whisper-large-v2","openai/whisper-large-v3"]

In [4]:
evaluateModel(whisper_path[0])

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

84.61538461538461
['杞人嘅朋友嘆咗一口氣', '聚星樓', '泥水佬開門口過得人過得自己', '嗰次喺跑馬地協和里見到有雞蛋仔格仔餅賣', '學生話佢知白沙山路係喺大棠 但係唔知有冇小巴經嗰度', '有個老人去左坪洲永安橫街食齋', '選定了心儀嘅機構先查詢個人實際年利率 咁會比較明智', '唔係', '正在做廉價勞工 收入少過做洗碗或保安員', '安哥拉']
['杞人休天嘅朋友探咗一口氣', '聚星路', '泥水路開門口過得人過得自己', '嗰次喺跑馬地鐵和里見到有雞蛋仔格仔餅民', '學生話佢知白沙山路係喺大堂但係唔知有冇小巴經嗰度', '有個老人去左坪洲永安橫街食齋', '選定了心義嘅機構先查詢個人實際連理率咁會比較明智', '唔係', '聖災做連家老公收入小歌做使碗環保安員', '噴個賴']


In [33]:
evaluateModel(whisper_path[1])

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

76.92307692307693
['杞人嘅朋友嘆咗一口氣', '聚星樓', '泥水佬開門口過得人過得自己', '嗰次喺跑馬地協和里見到有雞蛋仔格仔餅賣', '學生話佢知白沙山路係喺大棠 但係唔知有冇小巴經嗰度', '有個老人去左坪洲永安橫街食齋', '選定了心儀嘅機構先查詢個人實際年利率 咁會比較明智', '唔係', '正在做廉價勞工 收入少過做洗碗或保安員', '安哥拉']
['幾人休天嘅朋友嘆咗一口氣', '聚星流', '泥水路開門口過得人過得自己', '嗰次喺跑馬地協和里見到有雞蛋仔格仔餅賣', '學生話佢知白沙山路係喺大堂 但係唔知有冇小巴經嗰度', '有個老人去左坪洲永安環街食齋', '選定你好心怡既機構先查詢個人實際年厘率咁會比較明智', '唔係', '正在做年加勞工收入少個做世換環保安員', '本個拉']


In [7]:
evaluateModel(whisper_path[2])

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

1376.923076923077
['杞人嘅朋友嘆咗一口氣', '聚星樓', '泥水佬開門口過得人過得自己', '嗰次喺跑馬地協和里見到有雞蛋仔格仔餅賣', '學生話佢知白沙山路係喺大棠 但係唔知有冇小巴經嗰度', '有個老人去左坪洲永安橫街食齋', '選定了心儀嘅機構先查詢個人實際年利率 咁會比較明智', '唔係', '正在做廉價勞工 收入少過做洗碗或保安員', '安哥拉']
['a few people is good days friends had a good time', 'tresinglo', 'come open the door it is too much for you it is too much for yourself', '嗰次喺跑馬地協和里見到有雞蛋仔格仔餅賣', '學生話佢知白沙山路係喺大堂 但係唔知有冇小巴經嗰度', 'another老人去左坪洲永安環街食齋', '選定了心儀嘅機構先查詢個人實際年利率咁會比較明智', '唔係', 'ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස', 'ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස ස']


In [3]:
evaluateModel(whisper_path[2],"-4")

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

130.76923076923077
['杞人嘅朋友嘆咗一口氣', '聚星樓', '泥水佬開門口過得人過得自己', '嗰次喺跑馬地協和里見到有雞蛋仔格仔餅賣', '學生話佢知白沙山路係喺大棠 但係唔知有冇小巴經嗰度', '有個老人去左坪洲永安橫街食齋', '選定了心儀嘅機構先查詢個人實際年利率 咁會比較明智', '唔係', '正在做廉價勞工 收入少過做洗碗或保安員', '安哥拉']
['紀人 歐天 嘅朋友 嘆咗 一口氣', 'tui xin loh', '淋水滷開門口過得人過得自己', '嗰次喺跑馬地怯和里見到有雞蛋仔格仔餅賣', '學生話佢知白沙山路係喺大堂 但係唔知有冇小巴經嗰度', '有個老人去左坪洲永安環街食齋', '選定了深怡嘅機構先查詢個人實際連利率咁會比較明智', '唔係', '星載造亂加老公收入少個早晒碗滑步安員', '一個讚']


In [ ]:
evaluateModel(whisper_path[3])

In [ ]:
evaluateModel(whisper_path[4])

In [ ]:
evaluateModel(whisper_path[5])

In [ ]:
evaluateModel(whisper_path[6])